# **Clarifai as destination connector using Unstructured.IO**

This Notebook walk you through the step by step guide on how to utilise Clarifai as your destination connector using Unstructured.IO and import raw files from various source connectors. For the demo we will be using Dropbox as our source connector which has raw text files.

### Setup
Install necessary libraries.

In [ ]:
! pip install "unstructured[clarifai]" #make sure the unstructured version is 0.13 or above

In [ ]:
!pip install "unstructured[dropbox]"
!pip install httpx

## Dropbox access token

In [ ]:
DROPBOX_ACCESS_TOKEN="sl*********************************************************"

## Clarifai and dropbox imports

In [ ]:
import os
from unstructured.ingest.connector.fsspec.dropbox import DropboxAccessConfig, SimpleDropboxConfig
from unstructured.ingest.interfaces import (
    PartitionConfig,
    ProcessorConfig,
    ReadConfig,
)
from unstructured.ingest.runner import DropboxRunner

from unstructured.ingest.connector.clarifai import (
    ClarifaiAccessConfig,
    ClarifaiWriteConfig,
    SimpleClarifaiConfig,
)

from unstructured.ingest.runner.writers.base_writer import Writer
from unstructured.ingest.runner.writers.clarifai import (
    ClarifaiWriter,
)

## Clarifai_writer func which will configure the target clarifai app where the ingested documents will be loaded with initialising clarifai PAT as `api_key`

In [ ]:
def clarifai_writer() -> Writer:
    return ClarifaiWriter(
        connector_config=SimpleClarifaiConfig(
            access_config=ClarifaiAccessConfig(
                api_key= "YOUR_CLARIFAI_PAT"
            ),
            app_id= "YOUR_CLARIFAI_APP_ID",
            user_id= "YOUR_CLARIFAI_USER_ID"
            ),
        write_config=ClarifaiWriteConfig()
    )

## Let's now package the writer and runner together, make sure you have your dropbox URL ready.

In [ ]:
if __name__ == "__main__":
    writer = clarifai_writer()
    runner = DropboxRunner(
        processor_config=ProcessorConfig(
            verbose=True,
            output_dir="dropbox-output",
            num_processes=2,
        ),
        read_config=ReadConfig(),
        partition_config=PartitionConfig(),
        connector_config=SimpleDropboxConfig(
            access_config=DropboxAccessConfig(token=DROPBOX_ACCESS_TOKEN),
            remote_url="dropbox://test",
            recursive=True,
        ),
        writer=writer,
        writer_kwargs={},
    )
    runner.run()

## Let's do RAG over ingested documents from dropbox in just less than 4 lines of code.

In [ ]:
import os
#Replace your PAT
os.environ['CLARIFAI_PAT'] = "YOUR_CLARIFAI_PAT"

In [ ]:
from clarifai.rag import RAG
rag_agent = RAG.setup(app_url="YOUR CLARIFAI APP URL",
                      llm_url = "https://clarifai.com/mistralai/completion/models/mistral-large") # visit clarifai community to get more details of other available models.

In [ ]:
result=rag_agent.chat(messages=[{"role":"human", "content":"How to check the tire pressure?"}])
(result[0]["content"].split('\n'))


["Sure, I'd be happy to help you understand how to check tire pressure based on the information provided. Here are the steps:",
 '',
 '1. **Prepare your tire pressure gauge**: You can purchase a tire pressure gauge at auto parts stores or service stations. They are usually about the size of a large pen, but can come in other forms. Some gas stations also have a gauge built into their air pumps.',
 '',
 "2. **Remove the valve cap**: Each tire on your vehicle has a valve stem. It's a small black or chrome-colored tube that protrudes from the tire. At the end of the valve stem, there's a cap that you'll need to remove.",
 '',
 "3. **Align the gauge with the valve**: Place the end of your tire pressure gauge onto the valve stem. You want to ensure it's aligned correctly to get an accurate reading.",
 '',
 '4. **Press the gauge onto the valve**: Apply firm, direct pressure to the valve stem with the gauge. You should hear a brief hiss of air escaping, which is normal.',
 '',
 "5. **Read the